In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from tensorflow.keras.utils import get_file
from itertools import combinations
import matplotlib.pyplot as plt

In [78]:
movies_df = pd.read_csv("D:/추천 시스템/movies.csv", index_col=0)
genres_df = pd.read_csv("D:/추천 시스템/genres.csv", index_col=0)
ratings_df = pd.read_csv("D:/추천 시스템/ratings.csv", index_col=0)

id2title = dict(zip(movies_df["id"], movies_df["title"]))
ratings_df["title"] = ratings_df["movie_id"].map(id2title)

C:\Users\5CG7092POZ\Anaconda3\envs\tf2.0\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [79]:
oneday_id = movies_df[movies_df["title"]=="One Day"]["id"].values[0]

In [166]:
genres_set = genres_df.groupby(["movie_id"])["genre"].apply(set)

In [167]:
genres_set

movie_id
1         {Fantasy, Comedy, Adventure, Animation, Children}
2                            {Adventure, Fantasy, Children}
3                                         {Romance, Comedy}
4                                  {Romance, Drama, Comedy}
5                                                  {Comedy}
                                ...                        
131254                                             {Comedy}
131256                                             {Comedy}
131258                                          {Adventure}
131260                                 {(no genres listed)}
131262                         {Adventure, Fantasy, Horror}
Name: genre, Length: 27278, dtype: object

In [178]:
oneday_genre = genres_set[oneday_id]

same_genre = genres_set[genres_set==oneday_genre].index
same_genre = movies_df[movies_df["id"].isin(same_genre)]["title"]

In [179]:
same_genre

16                 Sense and Sensibility
24                     Leaving Las Vegas
27                            Persuasion
34                            Carrington
45         How to Make an American Quilt
                      ...               
27047                             Melody
27100                        Black Field
27104    The Suspended Step of the Stork
27160                             Bolero
27256             Por un puñado de besos
Name: title, Length: 1075, dtype: object

In [175]:
target = {8957: "Saw", 2232: "Cube", 6323: "Identity", 8638: "Before Sunset", 215: "Before Sunrise"}

In [183]:
df1 = movies_df[movies_df["id"].isin(target)]
df2= genres_df[genres_df["movie_id"].isin(target)]

pd.merge(df1, df2, left_on="id", right_on="movie_id")

,id,title,release_year,movie_id,genre
0,215,Before Sunrise,1995,215,Drama
1,215,Before Sunrise,1995,215,Romance
2,2232,Cube,1997,2232,Horror
3,2232,Cube,1997,2232,Mystery
4,2232,Cube,1997,2232,Sci-Fi
5,2232,Cube,1997,2232,Thriller
6,6323,Identity,2003,6323,Crime
7,6323,Identity,2003,6323,Horror
8,6323,Identity,2003,6323,Mystery
9,6323,Identity,2003,6323,Thriller


In [93]:
target_ratings = ratings_df[ratings_df["movie_id"].isin(target.keys())]

In [94]:
target_ratings

,user_id,movie_id,rating,rated_at,title
1099,11,2232,4.5,1230788590,Cube
3638,31,8957,0.5,1424736694,Saw
4977,50,215,5.0,1182347856,Before Sunrise
5058,50,8638,5.0,1182348573,Before Sunset
6156,56,215,3.0,1003025941,Before Sunrise
...,...,...,...,...,...
19997430,138472,8957,3.0,1267119268,Saw
19997639,138473,2232,4.5,1274234130,Cube
19998940,138483,215,4.0,833831946,Before Sunrise
19999482,138486,8957,5.0,1207076826,Saw


In [95]:
gby = target_ratings.groupby("user_id")
target_ratings_set = pd.DataFrame(gby["title"].apply(set))
all_seen = target_ratings_set[target_ratings_set.apply(lambda x : len(x["title"])==5, axis=1)].index
all_seen_df = target_ratings[target_ratings["user_id"].isin(all_seen)]

In [96]:
all_seen_df

,user_id,movie_id,rating,rated_at,title
146330,982,215,3.5,1195746062,Before Sunrise
146993,982,2232,3.5,1141652908,Cube
147899,982,6323,3.0,1193323976,Identity
148143,982,8638,4.0,1195747667,Before Sunset
148181,982,8957,3.0,1141659791,Saw
...,...,...,...,...,...
19885812,137686,215,4.0,899415564,Before Sunrise
19886569,137686,2232,4.0,1087503954,Cube
19887442,137686,6323,3.5,1079896026,Identity
19887645,137686,8638,4.0,1151393086,Before Sunset


In [97]:
ui = pd.pivot_table(all_seen_df, index="user_id", columns="title", values="rating")

In [108]:
avg_rating = ui.mean().mean()

gby_user = ratings_df.groupby(["user_id"]).mean()
mean_by_user = gby_user[gby_user.index.isin(all_seen)]["rating"]

gby_item = ratings_df.groupby(["title"]).mean()
mean_by_item = gby_item[gby_item.index.isin(target.values())]["rating"]

adj_ui = ui.sub(mean_by_user, axis=0).sub(mean_by_item, axis=1) + avg_rating

In [ ]:
# avg_rating = ui.mean().mean()

# user_bias = ui.mean(axis=1) - avg_rating
# item_bias = ui.mean(axis=0) - avg_rating

# adj_ui2 = ui.sub(user_bias, axis=0).sub(item_bias, axis=1)

In [146]:
adj_ui

title,Before Sunrise,Before Sunset,Cube,Identity,Saw
user_id,,,,,
982,0.083139,0.484533,0.329112,-0.161556,-0.044943
1165,1.035396,0.436790,0.781368,-0.709299,-0.592686
1288,0.259487,-0.339119,0.005460,-0.485208,0.631405
2650,-0.159526,-0.258132,-0.913554,0.095779,0.712392
2669,0.183951,0.085345,-0.070076,0.439256,0.055869
...,...,...,...,...,...
133329,1.068501,0.469895,1.314473,-0.676194,0.440419
134290,0.297501,0.698895,0.543474,-0.447194,0.169419
135071,0.049864,-0.048742,0.295836,0.305169,0.421782


In [158]:
sample5 = adj_ui.sample(5)

In [159]:
sample5

title,Before Sunrise,Before Sunset,Cube,Identity,Saw
user_id,,,,,
123601,1.584106,1.485499,-1.669922,-0.660589,-2.543976
3907,0.491599,0.892992,0.237571,-0.253096,-1.136484
74241,1.468190,1.369584,1.714162,1.723495,0.840108
80185,0.832375,0.733768,0.578347,0.087680,0.204293
43618,1.064639,0.466033,-0.189389,-0.180056,-1.063443


In [160]:
def mht_sim(x, y):
    return 1/(1 + np.sum(np.abs(x - y)))

for user1, user2 in combinations(sample5.index, 2):
    print(f"user1 : {user1:>6d} | user2 : {user2:>6d} | mht_sim : {mht_sim(sample5[sample5.index==user1].values, sample5[sample5.index==user2].values):>4.3f}")

user1 : 123601 | user2 :   3907 | mht_sim : 0.156
user1 : 123601 | user2 :  74241 | mht_sim : 0.096
user1 : 123601 | user2 :  80185 | mht_sim : 0.121
user1 : 123601 | user2 :  43618 | mht_sim : 0.167
user1 :   3907 | user2 :  74241 | mht_sim : 0.127
user1 :   3907 | user2 :  80185 | mht_sim : 0.284
user1 :   3907 | user2 :  43618 | mht_sim : 0.389
user1 :  74241 | user2 :  80185 | mht_sim : 0.176
user1 :  74241 | user2 :  43618 | mht_sim : 0.125
user1 :  80185 | user2 :  43618 | mht_sim : 0.263


In [161]:
def euc_sim(x, y):
    return 1/(1 + np.sqrt(np.sum((x - y)**2)))

for user1, user2 in combinations(sample5.index, 2):
    print(f"user1 : {user1:>6d} | user2 : {user2:>6d} | euc_sim : {euc_sim(sample5[sample5.index==user1].values, sample5[sample5.index==user2].values):>4.3f}")

user1 : 123601 | user2 :   3907 | euc_sim : 0.270
user1 : 123601 | user2 :  74241 | euc_sim : 0.157
user1 : 123601 | user2 :  80185 | euc_sim : 0.209
user1 : 123601 | user2 :  43618 | euc_sim : 0.291
user1 :   3907 | user2 :  74241 | euc_sim : 0.230
user1 :   3907 | user2 :  80185 | euc_sim : 0.404
user1 :   3907 | user2 :  43618 | euc_sim : 0.544
user1 :  74241 | user2 :  80185 | euc_sim : 0.305
user1 :  74241 | user2 :  43618 | euc_sim : 0.225
user1 :  80185 | user2 :  43618 | euc_sim : 0.393


In [162]:
def cos_sim(x, y):
    return np.sum(x*y)/(np.sqrt(np.sum(x**2))*np.sqrt(np.sum(y**2)))

for user1, user2 in combinations(sample5.index, 2):
    print(f"user1 : {user1:>6d} | user2 : {user2:>6d} | cos_sim : {cos_sim(sample5[sample5.index==user1].values, sample5[sample5.index==user2].values):>4.3f}")

user1 : 123601 | user2 :   3907 | cos_sim : 0.802
user1 : 123601 | user2 :  74241 | cos_sim : -0.144
user1 : 123601 | user2 :  80185 | cos_sim : 0.179
user1 : 123601 | user2 :  43618 | cos_sim : 0.910
user1 :   3907 | user2 :  74241 | cos_sim : 0.188
user1 :   3907 | user2 :  80185 | cos_sim : 0.476
user1 :   3907 | user2 :  43618 | cos_sim : 0.859
user1 :  74241 | user2 :  80185 | cos_sim : 0.854
user1 :  74241 | user2 :  43618 | cos_sim : 0.129
user1 :  80185 | user2 :  43618 | cos_sim : 0.436
